# Nopal Detector con YOLOv11 🌵 + Personas

Notebook optimizado para entrenamiento y predicciones en imágenes y video.

## 🎯 Objetivo
- Entrenar un modelo YOLOv11 para detectar nopales
- Detectar personas usando modelo preentrenado
- Procesar imágenes y videos con detecciones duales

## 🏗️ Arquitectura
Este notebook utiliza una arquitectura modular con clases especializadas:
- `DatasetManager`: Gestión del dataset de Roboflow
- `NopalPersonDetector`: Entrenamiento y predicciones
- `ResultVisualizer`: Visualización de resultados

## 📊 Flujo de Trabajo
1. **Instalación** de dependencias
2. **Descarga** del dataset desde Roboflow
3. **Preparación** y validación del dataset
4. **Entrenamiento** del modelo YOLOv11
5. **Predicciones** en imágenes de test
6. **Procesamiento** de video
7. **Descarga** de resultados

## 1. Instalación de Librerías Necesarias 🛠️

Instalamos las librerías requeridas para el proyecto:

In [ ]:
# Instalación de librerías necesarias
!pip install roboflow ultralytics supervision opencv-python PyYAML matplotlib -q

print("✅ Librerías instaladas correctamente")

In [ ]:
# Importar librerías necesarias
import os
import sys
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Any

# Agregar el directorio src al path para importar nuestras clases
sys.path.append('../src')

print("📚 Librerías importadas correctamente")

## 2. Configuración del Proyecto ⚙️

Cargar la configuración y inicializar las clases necesarias:

In [ ]:
# Cargar configuración del proyecto
import os
try:
    from utils.config import load_config_with_env, setup_environment
    print("✅ Utilidades de configuración importadas")
    
    # Configurar entorno
    setup_environment()
    
    # Cargar configuración
    CONFIG_PATH = "../config/model_config.yaml"
    if os.path.exists(CONFIG_PATH):
        config = load_config_with_env(CONFIG_PATH)
        print("✅ Configuración cargada con variables de entorno")
    else:
        raise FileNotFoundError(f"No se encontró {CONFIG_PATH}")
        
except ImportError:
    print("⚠️ No se pudieron importar utilidades, usando configuración manual")
    
    # Cargar variables de entorno manualmente
    try:
        from dotenv import load_dotenv
        load_dotenv("../.env")
        print("✅ Variables de entorno cargadas desde .env")
    except ImportError:
        print("⚠️ python-dotenv no disponible, usando variables del sistema")
    
    # Verificar API key
    roboflow_api_key = os.getenv('ROBOFLOW_API_KEY')
    if not roboflow_api_key:
        print("❌ ROBOFLOW_API_KEY no encontrada!")
        print("   1. Copia .env.example como .env")
        print("   2. Completa ROBOFLOW_API_KEY con tu API key")
        print("   3. O exporta: export ROBOFLOW_API_KEY=tu_api_key")
        roboflow_api_key = input("Ingresa tu API key de Roboflow: ")
    
    # Configuración para Colab y local
    if 'google.colab' in sys.modules:
        print("🔍 Detectado Google Colab")
        # En Colab, configurar variables de entorno
        os.environ['ROBOFLOW_API_KEY'] = roboflow_api_key
        
        config = {
            'roboflow': {
                'api_key': roboflow_api_key,
                'workspace': os.getenv('ROBOFLOW_WORKSPACE', 'nopaldetector'),
                'project': os.getenv('ROBOFLOW_PROJECT', 'nopal-detector-0lzvl'),
                'version': int(os.getenv('ROBOFLOW_VERSION', '2')),
                'format': 'yolov11'
            },
            'model': {
                'base_model': 'yolo11s.pt',
                'person_model': 'yolo11s.pt',
                'training': {
                    'epochs': 50,
                    'batch_size': 16,
                    'image_size': 640,
                    'patience': 10
                },
                'prediction': {
                    'confidence_threshold': float(os.getenv('MODEL_CONFIDENCE_THRESHOLD', '0.3')),
                    'iou_threshold': float(os.getenv('MODEL_IOU_THRESHOLD', '0.5'))
                }
            },
            'data': {
                'validation_split': 0.2,
                'random_seed': 42
            },
            'output': {
                'predictions_dir': '/content/predictions',
                'videos_dir': '/content/videos',
                'weights_dir': '/content/weights'
            }
        }
    else:
        # Configuración local
        config = {
            'roboflow': {
                'api_key': roboflow_api_key,
                'workspace': os.getenv('ROBOFLOW_WORKSPACE', 'nopaldetector'),
                'project': os.getenv('ROBOFLOW_PROJECT', 'nopal-detector-0lzvl'),
                'version': int(os.getenv('ROBOFLOW_VERSION', '2')),
                'format': 'yolov11'
            },
            'model': {
                'base_model': 'yolo11s.pt',
                'person_model': 'yolo11s.pt',
                'training': {
                    'epochs': 50,
                    'batch_size': 16,
                    'image_size': 640,
                    'patience': 10
                },
                'prediction': {
                    'confidence_threshold': float(os.getenv('MODEL_CONFIDENCE_THRESHOLD', '0.3')),
                    'iou_threshold': float(os.getenv('MODEL_IOU_THRESHOLD', '0.5'))
                }
            },
            'data': {
                'validation_split': 0.2,
                'random_seed': 42
            },
            'output': {
                'predictions_dir': '../outputs/predictions',
                'videos_dir': '../outputs/videos',
                'weights_dir': '../models/weights'
            }
        }

print("🎯 Configuración lista")
print(f"📊 Épocas de entrenamiento: {config['model']['training']['epochs']}")
print(f"🎨 Tamaño de imagen: {config['model']['training']['image_size']}")
print(f"🎯 Umbral de confianza: {config['model']['prediction']['confidence_threshold']}")
print(f"🔑 API key configurada: {'✅' if config['roboflow']['api_key'] else '❌'}")

## 3. Descarga de Dataset desde Roboflow 🗂️

Conectamos con la API de Roboflow y descargamos el dataset:

In [ ]:
# Importar y usar DatasetManager
try:
    from data.dataset_manager import DatasetManager
    print("✅ DatasetManager importado correctamente")
except ImportError:
    print("⚠️ No se pudo importar DatasetManager, usando implementación directa")
    
    # Implementación directa para Colab
    from roboflow import Roboflow
    import shutil
    import random
    
    # Descargar dataset
    rf = Roboflow(api_key=config['roboflow']['api_key'])
    project = rf.workspace(config['roboflow']['workspace']).project(config['roboflow']['project'])
    version = project.version(config['roboflow']['version'])
    dataset = version.download(config['roboflow']['format'])
    
    dataset_location = dataset.location
    print(f"✅ Dataset descargado en: {dataset_location}")
else:
    # Usar clase DatasetManager
    dataset_manager = DatasetManager(config)
    dataset_location = dataset_manager.download_dataset()
    
print(f"📂 Ubicación del dataset: {dataset_location}")

## 4. Preparación y Validación del Dataset 📂

Organizamos las carpetas del dataset y creamos el split de validación:

In [ ]:
# Preparar dataset
if 'dataset_manager' in locals():
    # Usar DatasetManager
    dataset_paths = dataset_manager.prepare_dataset()
    data_yaml_path = dataset_manager.get_data_yaml_path()
else:
    # Implementación directa
    base_dir = dataset_location
    
    # Definir rutas
    train_img_dir = os.path.join(base_dir, "train/images")
    train_lbl_dir = os.path.join(base_dir, "train/labels")
    valid_img_dir = os.path.join(base_dir, "valid/images")
    valid_lbl_dir = os.path.join(base_dir, "valid/labels")
    test_img_dir = os.path.join(base_dir, "test/images")
    test_lbl_dir = os.path.join(base_dir, "test/labels")
    
    # Crear todas las carpetas si no existen
    for d in [train_img_dir, train_lbl_dir, valid_img_dir, valid_lbl_dir, test_img_dir, test_lbl_dir]:
        os.makedirs(d, exist_ok=True)
    
    # Si no hay validación, crear un 20% de split desde train
    if not os.listdir(valid_img_dir):
        all_imgs = [f for f in os.listdir(train_img_dir) if f.lower().endswith((".jpg",".jpeg",".png"))]
        if all_imgs:
            random.seed(config['data']['random_seed'])
            split_size = max(1, int(len(all_imgs) * config['data']['validation_split']))
            sampled = random.sample(all_imgs, split_size)
            
            for img in sampled:
                lbl = os.path.splitext(img)[0] + ".txt"
                shutil.move(os.path.join(train_img_dir, img), os.path.join(valid_img_dir, img))
                if os.path.exists(os.path.join(train_lbl_dir, lbl)):
                    shutil.move(os.path.join(train_lbl_dir, lbl), os.path.join(valid_lbl_dir, lbl))
            
            print(f"⚠️ No había validación, se movieron {len(sampled)} imágenes a valid/")
        else:
            print("⚠️ No hay imágenes en train para dividir en valid.")
    
    # Actualizar data.yaml
    data_yaml_path = os.path.join(base_dir, "data.yaml")
    with open(data_yaml_path, "r") as f:
        data_yaml = yaml.safe_load(f)
    
    data_yaml["train"] = train_img_dir
    data_yaml["val"] = valid_img_dir
    
    # Solo incluir test si hay imágenes
    if any(f.lower().endswith((".jpg",".jpeg",".png")) for f in os.listdir(test_img_dir)):
        data_yaml["test"] = test_img_dir
    else:
        if "test" in data_yaml:
            del data_yaml["test"]
        print("⚠️ No se encontró carpeta test/ con imágenes, se omitirá del data.yaml")
    
    with open(data_yaml_path, "w") as f:
        yaml.dump(data_yaml, f)
    
    print(f"✅ data.yaml actualizado: {data_yaml_path}")

# Verificar estructura del dataset
print("📊 Estructura del dataset:")
with open(data_yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)
    
print(f"  📁 Train: {len(os.listdir(data_yaml['train']))} imágenes")
print(f"  📁 Valid: {len(os.listdir(data_yaml['val']))} imágenes")
if "test" in data_yaml:
    print(f"  📁 Test: {len(os.listdir(data_yaml['test']))} imágenes")
print(f"  🏷️ Clases: {data_yaml['names']}")

## 5. Entrenamiento del Modelo YOLOv11 🤖🏋️

Entrenamos el modelo con nuestro dataset de nopales:

In [ ]:
# Entrenar modelo usando NopalPersonDetector o implementación directa
try:
    from models.detector import NopalPersonDetector
    print("✅ NopalPersonDetector importado correctamente")
    
    # Crear detector y entrenar
    detector = NopalPersonDetector(config)
    results = detector.train_nopal_model(data_yaml_path)
    
except ImportError:
    print("⚠️ No se pudo importar NopalPersonDetector, usando implementación directa")
    
    # Implementación directa
    from ultralytics import YOLO
    
    print("🤖 Iniciando entrenamiento del modelo de nopales...")
    
    # Cargar modelo base
    model = YOLO(config['model']['base_model'])
    
    # Configuración de entrenamiento
    train_config = config['model']['training']
    
    # Entrenar
    results = model.train(
        data=data_yaml_path,
        epochs=train_config['epochs'],
        imgsz=train_config['image_size'],
        batch=train_config['batch_size'],
        patience=train_config['patience']
    )
    
    print("✅ Entrenamiento completado")

# Mostrar información del entrenamiento
print("📊 Resumen del entrenamiento:")
print(f"  ⏱️ Épocas completadas: {train_config['epochs']}")
print(f"  🖼️ Tamaño de imagen: {train_config['image_size']}px")
print(f"  📦 Tamaño de lote: {train_config['batch_size']}")
print(f"  🎯 Paciencia: {train_config['patience']}")

## 6. Predicciones en Imágenes de Test 🖼️ (Nopales + Personas)

Aplicamos el modelo entrenado para detectar nopales y el modelo preentrenado para detectar personas:

In [ ]:
# Cargar modelos y realizar predicciones en imágenes
if 'detector' in locals():
    # Usar NopalPersonDetector
    detector.load_models()
    
    # Verificar si hay imágenes de test
    with open(data_yaml_path, "r") as f:
        data_yaml = yaml.safe_load(f)
    
    if "test" in data_yaml:
        test_img_dir = data_yaml["test"]
        if os.path.exists(test_img_dir) and os.listdir(test_img_dir):
            predictions_dir = detector.predict_images(test_img_dir)
            stats = detector.get_detection_stats(test_img_dir)
            print(f"📊 Estadísticas: {stats}")
        else:
            print("⚠️ No hay imágenes de test disponibles")
    else:
        print("⚠️ Dataset no incluye carpeta test")
        
else:
    # Implementación directa
    from ultralytics import YOLO
    
    # Determinar rutas según entorno
    if 'google.colab' in sys.modules:
        best_model_path = "/content/runs/detect/train/weights/best.pt"
        predictions_dir = config['output']['predictions_dir']
    else:
        best_model_path = "runs/detect/train/weights/best.pt"
        predictions_dir = config['output']['predictions_dir']
    
    os.makedirs(predictions_dir, exist_ok=True)
    
    # Cargar data.yaml
    with open(data_yaml_path, "r") as f:
        data_yaml = yaml.safe_load(f)
    
    if "test" in data_yaml:
        test_img_dir = data_yaml["test"]
        if os.path.exists(test_img_dir) and os.listdir(test_img_dir):
            print(f"📂 Imágenes de test encontradas en: {test_img_dir}")
            
            if os.path.exists(best_model_path):
                nopal_model = YOLO(best_model_path)
                person_model = YOLO(config['model']['person_model'])
                
                conf_thresh = config['model']['prediction']['confidence_threshold']
                
                res_nopal = nopal_model(test_img_dir, save=False, conf=conf_thresh)
                res_person = person_model(test_img_dir, save=False, conf=conf_thresh)
                
                total_nopales = 0
                total_persons = 0
                
                for idx, r_nopal in enumerate(res_nopal):
                    img_path = r_nopal.path
                    img = cv2.imread(img_path)
                    if img is None:
                        continue
                    
                    annotated_img = img.copy()
                    
                    # Dibujar detecciones de nopales (verde)
                    if r_nopal.boxes is not None:
                        total_nopales += len(r_nopal.boxes)
                        for box in r_nopal.boxes:
                            x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
                            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            if box.conf is not None:
                                label = f"nopal: {box.conf.item():.2f}"
                                cv2.putText(annotated_img, label, (x1, y1 - 10), 
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    # Dibujar detecciones de personas (azul)
                    if res_person[idx].boxes is not None:
                        for box in res_person[idx].boxes:
                            if int(box.cls) == 0:  # Solo personas (clase 0 en COCO)
                                total_persons += 1
                                x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
                                cv2.rectangle(annotated_img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                                if box.conf is not None:
                                    label = f"person: {box.conf.item():.2f}"
                                    cv2.putText(annotated_img, label, (x1, y1 - 10), 
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                    
                    # Guardar imagen anotada
                    out_path = os.path.join(predictions_dir, os.path.basename(img_path))
                    cv2.imwrite(out_path, annotated_img)
                
                print(f"📌 Guardadas {len(res_nopal)} imágenes con detecciones")
                print(f"🌵 Total nopales detectados: {total_nopales}")
                print(f"👥 Total personas detectadas: {total_persons}")
                
            else:
                print(f"⚠️ No se encontró el modelo entrenado en: {best_model_path}")
        else:
            print("⚠️ No hay imágenes de test para evaluar")
    else:
        print("⚠️ El dataset no incluye test/")

print(f"✅ Resultados guardados en: {predictions_dir}")

## 7. Procesamiento de Video con Detecciones 🎥

Procesamos un video frame por frame aplicando detección dual:

In [ ]:
# Procesamiento de video
if 'google.colab' in sys.modules:
    video_filename = "videoEjemplo.mp4"
    video_path = os.path.join("/content", video_filename)
    output_video_path = "/content/output_video.mp4"
else:
    # Para entorno local
    video_filename = input("Ingrese el nombre del archivo de video (ej: video.mp4): ") or "video.mp4"
    video_path = os.path.join("../data", video_filename)
    output_video_path = os.path.join(config['output']['videos_dir'], "output_video.mp4")
    os.makedirs(config['output']['videos_dir'], exist_ok=True)

if not os.path.exists(video_path):
    print(f"⚠️ No se encontró el archivo en {video_path}")
    if 'google.colab' in sys.modules:
        print("📁 Por favor, sube tu video a Colab con el nombre 'videoEjemplo.mp4'")
    else:
        print(f"📁 Por favor, coloca tu video en la carpeta 'data/' con el nombre '{video_filename}'")
else:
    if 'detector' in locals():
        # Usar NopalPersonDetector
        output_path = detector.process_video(video_path, "output_video.mp4")
        print(f"✅ Video procesado guardado en: {output_path}")
    else:
        # Implementación directa
        if 'google.colab' in sys.modules:
            best_model_path = "/content/runs/detect/train/weights/best.pt"
        else:
            best_model_path = "runs/detect/train/weights/best.pt"
            
        if os.path.exists(best_model_path):
            from ultralytics import YOLO
            
            nopal_model = YOLO(best_model_path)
            person_model = YOLO(config['model']['person_model'])
            print("✅ Modelos cargados (Nopal + Persona)")
            
            cap = cv2.VideoCapture(video_path)
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
            
            conf_thresh = config['model']['prediction']['confidence_threshold']
            frame_count = 0
            
            print("🎬 Procesando frames...")
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                res_nopal = nopal_model(frame, conf=conf_thresh, verbose=False)
                res_person = person_model(frame, conf=conf_thresh, verbose=False)
                
                annotated_frame = frame.copy()
                
                # Dibujar detecciones de nopales (verde)
                if res_nopal[0].boxes is not None:
                    for box in res_nopal[0].boxes:
                        x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
                        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        if box.conf is not None:
                            label = f"nopal: {box.conf.item():.2f}"
                            cv2.putText(annotated_frame, label, (x1, y1 - 10), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Dibujar detecciones de personas (azul)
                if res_person[0].boxes is not None:
                    for box in res_person[0].boxes:
                        if int(box.cls) == 0:  # Solo personas
                            x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
                            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                            if box.conf is not None:
                                label = f"person: {box.conf.item():.2f}"
                                cv2.putText(annotated_frame, label, (x1, y1 - 10), 
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                
                out.write(annotated_frame)
                frame_count += 1
                
                # Mostrar progreso cada 100 frames
                if frame_count % 100 == 0:
                    print(f"📹 Procesados {frame_count} frames...")
                    
                    # Mostrar preview en Colab
                    if 'google.colab' in sys.modules:
                        try:
                            from google.colab.patches import cv2_imshow
                            cv2_imshow(annotated_frame)
                        except:
                            pass
            
            cap.release()
            out.release()
            print(f"✅ Video procesado guardado en: {output_video_path}")
            
        else:
            print("⚠️ No se encontraron pesos entrenados. No se procesó el video.")

## 8. Visualización de Resultados 📊

Generamos gráficos y visualizaciones de los resultados:

In [ ]:
# Visualización de resultados usando ResultVisualizer
try:
    from utils.visualization import ResultVisualizer
    print("✅ ResultVisualizer importado correctamente")
    
    # Crear visualizador
    if 'google.colab' in sys.modules:
        viz_dir = "/content/visualizations"
    else:
        viz_dir = "../outputs/visualizations"
    
    visualizer = ResultVisualizer(viz_dir)
    
    # Generar gráficos de entrenamiento
    if 'google.colab' in sys.modules:
        training_plot = visualizer.plot_training_results("/content/runs/detect/train")
    else:
        training_plot = visualizer.plot_training_results("runs/detect/train")
    
    # Crear grilla de detecciones
    if 'predictions_dir' in locals() and os.path.exists(predictions_dir):
        grid_plot = visualizer.create_detection_grid(predictions_dir)
        
        # Estadísticas de detección
        if 'stats' in locals():
            stats_plot = visualizer.plot_detection_stats(stats)
        elif 'total_nopales' in locals():
            stats = {
                'total_images': len(os.listdir(predictions_dir)),
                'total_nopales': total_nopales,
                'total_persons': total_persons
            }
            stats_plot = visualizer.plot_detection_stats(stats)
    
    print("✅ Visualizaciones generadas correctamente")
    
except ImportError:
    print("⚠️ No se pudo importar ResultVisualizer, generando visualización básica")
    
    # Visualización básica con matplotlib
    if 'total_nopales' in locals():
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        
        categories = ['Nopales', 'Personas']
        counts = [total_nopales, total_persons]
        colors = ['green', 'blue']
        
        bars = ax.bar(categories, counts, color=colors, alpha=0.7)
        ax.set_title('Detecciones Totales', fontsize=14, fontweight='bold')
        ax.set_ylabel('Número de Detecciones')
        
        # Agregar valores en las barras
        for bar, count in zip(bars, counts):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(counts) * 0.01, 
                   str(count), ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        print(f"📊 Estadísticas finales:")
        print(f"  🌵 Nopales detectados: {total_nopales}")
        print(f"  👥 Personas detectadas: {total_persons}")
    
    else:
        print("⚠️ No hay estadísticas disponibles para visualizar")

## 9. Descarga de Resultados 💾

Descargamos los archivos generados (específico para Google Colab):

In [ ]:
# Descarga de resultados (solo en Google Colab)
if 'google.colab' in sys.modules:
    from google.colab import files
    import zipfile
    
    print("📦 Preparando archivos para descarga...")
    
    # Crear archivo ZIP con todos los resultados
    zip_filename = "/content/nopal_detector_results.zip"
    
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        # Agregar modelo entrenado si existe
        if os.path.exists("/content/runs/detect/train/weights/best.pt"):
            zipf.write("/content/runs/detect/train/weights/best.pt", "models/best.pt")
            print("✅ Modelo entrenado agregado al ZIP")
        
        # Agregar predicciones si existen
        if 'predictions_dir' in locals() and os.path.exists(predictions_dir):
            for file in os.listdir(predictions_dir):
                file_path = os.path.join(predictions_dir, file)
                if os.path.isfile(file_path):
                    zipf.write(file_path, f"predictions/{file}")
            print("✅ Predicciones agregadas al ZIP")
        
        # Agregar video procesado si existe
        if os.path.exists("/content/output_video.mp4"):
            zipf.write("/content/output_video.mp4", "videos/output_video.mp4")
            print("✅ Video procesado agregado al ZIP")
        
        # Agregar visualizaciones si existen
        if os.path.exists("/content/visualizations"):
            for file in os.listdir("/content/visualizations"):
                file_path = os.path.join("/content/visualizations", file)
                if os.path.isfile(file_path):
                    zipf.write(file_path, f"visualizations/{file}")
            print("✅ Visualizaciones agregadas al ZIP")
    
    print(f"📁 Archivo ZIP creado: {zip_filename}")
    
    # Descargar archivos individuales
    print("\n🔽 Iniciando descargas...")
    
    # Descargar video procesado
    if os.path.exists("/content/output_video.mp4"):
        print("📹 Descargando video procesado...")
        files.download("/content/output_video.mp4")
    
    # Descargar archivo ZIP completo
    print("📦 Descargando archivo ZIP con todos los resultados...")
    files.download(zip_filename)
    
    print("✅ ¡Descargas completadas!")
    
else:
    print("📁 Los resultados están guardados en las siguientes ubicaciones:")
    print(f"  🤖 Modelo entrenado: runs/detect/train/weights/best.pt")
    if 'predictions_dir' in locals():
        print(f"  🖼️ Predicciones: {predictions_dir}")
    if 'output_video_path' in locals():
        print(f"  🎥 Video procesado: {output_video_path}")
    if 'viz_dir' in locals():
        print(f"  📊 Visualizaciones: {viz_dir}")
    
    print("\n🎯 Para usar el modelo entrenado en el futuro:")
    print("   from ultralytics import YOLO")
    print("   model = YOLO('runs/detect/train/weights/best.pt')")
    print("   results = model('imagen.jpg')")

## 🎉 Resumen del Proyecto

### ✅ Tareas Completadas:
1. **Instalación** de todas las librerías necesarias
2. **Descarga** del dataset desde Roboflow 
3. **Preparación** automática del dataset con validación
4. **Entrenamiento** del modelo YOLOv11 para detección de nopales
5. **Predicciones** duales en imágenes (nopales + personas)
6. **Procesamiento** de video con detecciones en tiempo real
7. **Visualización** de resultados y estadísticas
8. **Descarga** de todos los archivos generados

### 🏗️ Arquitectura del Proyecto:
- **Modular**: Clases especializadas para cada funcionalidad
- **Escalable**: Fácil de extender con nuevas características
- **Portable**: Funciona tanto en local como en Google Colab
- **Configurable**: Parámetros centralizados en archivos YAML

### 📈 Próximos Pasos:
- Ajustar hiperparámetros para mejor rendimiento
- Agregar más clases de detección
- Implementar seguimiento de objetos en video
- Crear una interfaz web para el modelo

### 🚀 ¡Tu Detector de Nopales está listo para usar!